<a href="https://colab.research.google.com/github/bcataldo3/plantilla_curso_python/blob/main/docs/Clase_04 - Modelo K-Medias Caso Estudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Caso de Estudio

En este capítulo, estamos utilizando el conocido conjunto de datos Breast Cancer Wisconsin para realizar un análisis de clústeres. Originalmente, el conjunto de datos se propuso con el fin de entrenar clasificadores; sin embargo, también puede ser muy útil para un análisis de clúster no trivial. Contiene 569 registros compuestos por 32 atributos (incluyendo el diagnóstico y un número de identificación). Todos los atributos están estrictamente relacionados con propiedades biológicas y morfológicas de los tumores, pero nuestro objetivo es validar hipótesis genéricas considerando la verdad fundamental (benigna o maligna) y las propiedades estadísticas del conjunto de datos. Antes de continuar, es importante aclarar algunos puntos. El conjunto de datos es de alta dimensionalidad y los clústeres son no convexos (por lo que no podemos esperar una segmentación perfecta). Además, nuestro objetivo no es utilizar un algoritmo de agrupamiento para obtener los resultados de un clasificador; por lo tanto, la verdad fundamental debe tenerse en cuenta solo como una indicación genérica de un posible agrupamiento. El objetivo de este ejemplo es mostrar cómo realizar un breve análisis preliminar, seleccionar el número óptimo de clústeres y validar los resultados finales

In [ ]:
import pandas as pd

url='https://drive.google.com/file/d/1LG6hb0duLtnSiFhT9JWFNuo8qDxHV08W/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

dataset_columns = ['id','diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean','area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean','concave points_mean', 'symmetry_mean', 'fractal_dimension_mean','radius_se','texture_se', 'perimeter_se', 'area_se', 'smoothness_se','compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se','fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst','area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst','concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
df = pd.read_csv(url, sep="," ,index_col=0, names=dataset_columns)
print(df.describe())

Invitamos al lector a verificar los valores de todos los atributos, incluso si estamos enfocando nuestra atención solo en un subconjunto. Dejamos algunas observaciones acerca del manejo que tendremos de la data.

* En particular, debemos observar las diferentes escalas que existen entre los primeros ocho atributos. Las desviaciones estándar van desde 0.01 hasta 350, lo que significa que muchos vectores podrían ser extremadamente similares solo debido a uno o dos atributos. 
* Por otro lado, normalizar el valor con una escala de varianza dará a todos los atributos la misma responsabilidad (por ejemplo, area_mean está limitada entre 143.5 y 2501, mientras que smoothness_mean está limitada entre 0.05 y 0.16. Forzarlos a tener la misma varianza puede influir en el impacto biológico de los factores y, como no tenemos ninguna indicación específica, no estamos autorizados a tomar esa decisión). 
* Claramente, algunos atributos tendrán un mayor peso en el proceso de agrupación y aceptamos su influencia principal como una condición relacionada con el contexto

Revisaremos la diferencia en distribución de algunos de los atributos de la data (perimeter_mean, area_mean, smoothness_mean, concavity_mean, y symmetry_mean)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

# Show the pair-plot
sns.set()

sns.plotting_context("notebook", font_scale=1.2)
sns.pairplot(df,vars=['perimeter_mean', 'area_mean', 'smoothness_mean', 'concavity_mean', 'symmetry_mean'],hue="diagnosis")
plt.show()

Como sabemos, estos graficos nos muestran como se distribuye el atributo de diagnostico (benigno o maligno) entre cada par de variables seleccionadas. En la diagonal nos muestra la distribucion que tienen ambos valores de la variable diagnostico para una variable en particular. Algunas conclusiones de nuestro análisis preliminar podrían ser:
* "area_mean" y "perimeter_mean" tienen una clara correlación y determinan una separación abrupta. Cuando "area_mean" es mayor que aproximadamente 1,000, claramente el "perimeter" también aumenta, y el diagnóstico cambia de benigno a maligno de manera brusca. Por lo tanto, estos dos atributos son determinantes para el resultado final y es probable que uno de ellos sea redundante.

* Otras gráficas (por ejemplo, "perimeter_mean/area_mean" frente a "smoothness_mean", "area_mean" frente a "symmetry_mean", "concavity_mean" frente a "smoothness_mean" y "concavity_mean" frente a "symmetry_mean") muestran una separación horizontal (que se convierte en vertical al invertir el eje). Esto significa que, para casi todos los valores asumidos por la variable independiente (eje x), hay un umbral que separa los valores de la otra variable en dos conjuntos (benigno y maligno).

* Algunas gráficas (por ejemplo, "perimeter_mean/area_mean" frente a "concavity_mean" y "concavity_mean" frente a "symmetry_mean") muestran una ligera separación diagonal con pendiente negativa. Esto significa que cuando la variable independiente es pequeña, el diagnóstico permanece constante para casi todos los valores de la variable dependiente, mientras que, por otro lado, cuando la variable independiente se vuelve más grande, el diagnóstico cambia proporcionalmente al valor opuesto. Por ejemplo, para valores pequeños de "perimeter_mean", "concavity_mean" puede alcanzar su máximo sin afectar el diagnóstico (que es benigno), mientras que un "perimeter_mean" > 150 siempre resulta en un diagnóstico maligno independientemente de "concavity_mean".

Por supuesto, no podemos sacar fácilmente nuestras conclusiones de un análisis dividido (porque debemos considerar todas las interacciones), pero esta actividad será útil para proporcionar a cada grupo un etiquetado semántico

### t-Distributed Stochastic Neighbor Embedding (t-SNE)

t-SNE (t-distributed Stochastic Neighbor Embedding) es una técnica de reducción de dimensionalidad no lineal no supervisada utilizada para explorar datos y visualizar datos de alta dimensionalidad. La reducción de dimensionalidad no lineal significa que el algoritmo nos permite separar datos que no pueden ser separados por una línea recta.

t-distributed stochastic neighbor embedding (t-SNE) es un método estadístico para visualizar datos de alta dimensionalidad al asignar a cada punto de datos una ubicación en un mapa de dos o tres dimensiones. Es una técnica de reducción de dimensionalidad no lineal para representar datos de alta dimensionalidad en un espacio de baja dimensionalidad de dos o tres dimensiones. Específicamente, modela cada objeto de alta dimensionalidad mediante un punto de dos o tres dimensiones de tal manera que objetos similares se modelan mediante puntos cercanos y objetos disimilares se modelan mediante puntos distantes con alta probabilidad.

El algoritmo t-SNE consta de dos etapas principales. Primero, t-SNE construye una distribución de probabilidad sobre pares de objetos de alta dimensionalidad de tal manera que objetos similares tienen asignada una probabilidad más alta, mientras que los puntos disímiles tienen asignada una probabilidad más baja. Segundo, t-SNE define una distribución de probabilidad similar sobre los puntos en el mapa de baja dimensionalidad y minimiza la divergencia Kullback-Leibler (divergencia KL) entre las dos distribuciones con respecto a las ubicaciones de los puntos en el mapa. Si bien el algoritmo original utiliza la distancia euclidiana entre objetos como base de su métrica de similitud, esto se puede cambiar según sea necesario. Una variante riemanniana es UMAP.

t-SNE se ha utilizado para la visualización en una amplia gama de aplicaciones, incluyendo genómica, investigación en seguridad informática, procesamiento de lenguaje natural, análisis de música, investigación sobre el cáncer, bioinformática, interpretación de dominio geológico y procesamiento de señales biomédicas. Si bien las representaciones de t-SNE a menudo parecen mostrar grupos, los grupos visuales pueden estar fuertemente influenciados por la elección de la parametrización y, por lo tanto, es necesario comprender bien los parámetros de t-SNE. Estos "grupos" pueden incluso aparecer en datos no agrupados y, por lo tanto, pueden ser hallazgos falsos. La exploración interactiva puede ser necesaria para elegir parámetros y validar resultados. Se ha demostrado que t-SNE a menudo es capaz de recuperar grupos bien diferenciados y, con elecciones de parámetros especiales, se aproxima a una forma simple de agrupación espectral."

#### Visualizacion de Data con t-SNE

En este punto, es útil visualizar el conjunto de datos (sin los atributos no estructurales) en un plano bidimensional a través de una transformación de t-Distributed Stochastic Neighbor Embedding (t-SNE) (para más detalles, consulte "Visualizing Data using t-SNE", van der Maaten L., Hinton G., Journal of Machine Learning Research 9, 2008).

In [ ]:
from sklearn.manifold import TSNE
cdf = df.drop(['diagnosis'], axis=1)
tsne = TSNE(n_components=2, perplexity=10, random_state=1000)
data_tsne = tsne.fit_transform(cdf)
df_tsne = pd.DataFrame(data_tsne, columns=['x', 'y'], index=cdf.index)
dff = pd.concat([df, df_tsne], axis=1)